*Материал оформила и подготовила* Виктория Фирсанова

#  **Тонкая настройка GPT-2 для генерации текста**


In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## Доступные модели

Размер модели зависит от количества параметров (а количество настроек - от количества слоев). Для обучения использовалось 8 миллионов сайтов, 40 Гб текста. 

|Параметры | Слои | Соответствие |
| ---------| ---- |------------|
117M | 12 | Предшествующая GPT
345M | 24 | Самая большая модель BERT
762M | 36 | 
1542M | 48 | GPT-2

Варианты GPT-2, доступные для использования (отличаются от перечисленных в оригинальной статье):

*   117M
*   124M
*   355M
*   774M
*   1558M

Чем больше модель, тем больше времени вам потребуется на ее настройку (логично), но и качество генерации текста будет заметно выше.

Почему в таком случае сразу не перейти к GPT-3? Что ж, доступ к ней не так легко получить. Но вы можете попробовать: https://beta.openai.com/

## **Файн-тюнинг модели**

Сессия TensorFlow, в которой обучение проводится заданное количество шагов (можно задать бесконечное обучение `steps = -1`).

Параметры:

*  **`restore_from`**: запускаем `fresh`, чтобы начать обучение с базовой GPT-2, или `latest`, чтобы перезапустить обучение с контрольной точки.
* **`sample_every`**: количество шагов, спустя которое будет выведен пример работы модели на данном этапе обучения.
* **`print_every`**: количество шагов, спустя которого выводится прогресс обучения.
*  **`run_name`**: подпапка `checkpoint`, используется для сохранения модели. Полезно, если вы работаете с несколькими моделями. 
* **`overwrite`**: позволяет продолжать настройку существующей модели, не создавая дубликатов. 

In [ ]:
gpt2.download_gpt2(model_name="124M")

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=5000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

## **Запуск предобученной модели**

*  **`length`**: количество токенов для генерации (по умолчанию = 1023).
* **`temperature`**: чем выше этот параметр, тем безумнее получится текст (по умолчанию = 0.7).
* **`top_k`**: ограничивает сгенерированные предсказания до верхних *k* (0 отключает эту опцию - по умолчанию) - еще одно средство для обеспечения адекватности.
* **`top_p`**: обеспечивает кумулятивное распрделение предсказаний (рекомендованный размер = 0.9).
* **`truncate`**: обрезает инпут до заданного ограничения, функция полезна при работе с низкими значениями `length`.
*  **`include_prefix`**: дополнение к `truncate`, обозначенный `prefix` будет включен в вывод.

In [ ]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 258Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 128Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 393Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [00:49, 62.6Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 474Mit/s]                                                
Fetching model.ckpt.meta: 2.10Mit [00:00, 142Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 117Mit/s]                                                       


In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

Loading pretrained model models/774M/model.ckpt
INFO:tensorflow:Restoring parameters from models/774M/model.ckpt


In [ ]:
gpt2.generate(sess,
              model_name=model_name,
              prefix=" Hi! ",
              length=100,
              temperature=1,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
 Hi!  In II, I take one step back and examine our collective project and how it relates to our present.  Before discussing this case, it's useful to remind ourselves what I think of as our  OUPposture that fosters systematic self-deception.  If you read it through, you'll see that the surface of the articulation is a fixation on food (that we obtain, cook, and eat) and exercise, but that at the core of the post
 Hi! ~~~~~ O:O ~~~~~ D:O O:O In D:O I always cry... I always go abroad... I hope it's too late I think my fate is to be married in future to girls... Because this day I live by fear of not knowing... For getting down on myself, it's like I refuse to look at things and pursue them, it's like I won't take them into my hands... __________________________________________________________________________________________________________________________________________________________ When I op

**Список источников:**

1. Radford, A., Wu, J., Child, R., Luan, D., Amodei, D. & Sutskever, I. (2019). Language Models are Unsupervised Multitask Learners. Technical report, OpenAI
2. Woolf, M. How To Make Custom AI-Generated Text With GPT-2. (2019). Max Woolf's Blog. URL: https://minimaxir.com/2019/09/howto-gpt2/ 